In [32]:
import pandas as pd
import numpy as np
import re
import csv
import time
from datetime import datetime, timedelta
import snscrape.modules.twitter as sntwitter

In [33]:
def createQuery(word, acc, start, end):
    return word + " from:" + acc + " since:" + start + " until:" + end

In [34]:
accs = [
    'nytimesworld',
    'cnni',
    'BBCWorld',
    'Reuters'
]

words = [
    "russia",
    "ukraine",
    "putin",
    "zelensky",
    "war"]

bag = ' OR '.join(words)

bag = "(" + bag + ")"

tweets = []
wordLimitGlobal = 100

start = datetime(2022, 1, 1)
end = datetime(2023, 4, 30)
output_format = "%Y-%m-%d"

col_names =  ['Date', 'Tweet Treated', 'Tweet Raw', 'User', 'Url', 'Id', 'Likes', 'Views']

In [35]:
duration = end - start
duration = int(duration.days)

startAux = start


In [ ]:
wordLimit = wordLimitGlobal

for i in range(1, duration+1):
    endString = startAux + timedelta(days=1)
    endString = endString.strftime(output_format)
    
    for a in accs:
        query = createQuery(bag, a, startAux.strftime(output_format), endString)
        print("QUERY: " + query)
        
        for tweet in sntwitter.TwitterSearchScraper(query).get_items():
            no_link = re.sub(r'https?://\S+', '', tweet.rawContent.lower())
            no_link_no_user = re.sub(r'\@[\w\_]+', '', no_link)

            tweets.append([tweet.date, no_link_no_user, tweet.rawContent, a, tweet.url, tweet.id, tweet.likeCount, tweet.viewCount])
        
        time.sleep(.5)

    dfTemp = pd.DataFrame(tweets, columns=col_names)
    dfTemp = pd.DataFrame(dfTemp.drop_duplicates(subset=['Id'], keep='last'))
    dfTemp.to_csv("data_news/"+startAux.strftime(output_format)+".csv", index=False, quoting=csv.QUOTE_NONNUMERIC)

    tweets = []
    startAux = startAux + timedelta(days=1)

In [38]:
import os

dfAll = pd.DataFrame(columns=col_names)


path = "data_news/"
with os.scandir(path) as entries:
    for entry in entries:
        dfRunning = pd.read_csv("data_news/"+str(entry.name))
        dfAll = dfAll.append(dfRunning)

C:\Users\Alexandre\AppData\Local\Temp\ipykernel_10420\2741531107.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfAll = dfAll.append(dfRunning)
C:\Users\Alexandre\AppData\Local\Temp\ipykernel_10420\2741531107.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfAll = dfAll.append(dfRunning)
C:\Users\Alexandre\AppData\Local\Temp\ipykernel_10420\2741531107.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfAll = dfAll.append(dfRunning)
C:\Users\Alexandre\AppData\Local\Temp\ipykernel_10420\2741531107.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfAll = dfAll.append(dfRunning)
C:\Users\Alexandre\AppData\Local\Temp\ipyker

In [39]:
dfAll

,Date,Tweet Treated,Tweet Raw,User,Url,Id,Likes,Views
0,2022-01-01 18:54:29+00:00,president joe biden will speak with ukrainian ...,President Joe Biden will speak with Ukrainian ...,cnni,https://twitter.com/cnni/status/14773525459578...,1477352545957863430,163,NaN
1,2022-01-01 01:55:07+00:00,biden says he warned putin of 'heavy price' ov...,Biden says he warned Putin of 'heavy price' ov...,Reuters,https://twitter.com/Reuters/status/14770960141...,1477096014129602561,178,NaN
2,2022-01-01 01:05:08+00:00,biden says he made it clear to putin that russ...,Biden says he made it clear to Putin that Russ...,Reuters,https://twitter.com/Reuters/status/14770834331...,1477083433184731137,458,NaN
0,2022-01-02 21:00:25+00:00,in a region where the shadow of war is everywh...,In a region where the shadow of war is everywh...,nytimesworld,https://twitter.com/nytimesworld/status/147774...,1477746627641286661,8,NaN
1,2022-01-02 14:30:07+00:00,bosnia is facing its gravest crisis since the ...,Bosnia is facing its gravest crisis since the ...,nytimesworld,https://twitter.com/nytimesworld/status/147764...,1477648401785405440,13,NaN
...,...,...,...,...,...,...,...,...
4,2023-04-29 13:23:02+00:00,ukraine war: crimea oil tank set ablaze by rep...,Ukraine war: Crimea oil tank set ablaze by rep...,BBCWorld,https://twitter.com/BBCWorld/status/1652302472...,1652302472793210883,435,242940.0
5,2023-04-29 09:14:34+00:00,ukraine war: crimea oil tank set ablaze by rep...,Ukraine war: Crimea oil tank set ablaze by rep...,BBCWorld,https://twitter.com/BBCWorld/status/1652239945...,1652239945610739713,456,282937.0
6,2023-04-29 17:35:00+00:00,ukraine held military drills in an undisclosed...,Ukraine held military drills in an undisclosed...,Reuters,https://twitter.com/Reuters/status/16523658814...,1652365881467445248,82,76332.0
7,2023-04-29 04:30:00+00:00,after a year-long war of words between florida...,After a year-long war of words between Florida...,Reuters,https://twitter.com/Reuters/status/16521683298...,1652168329845436416,109,71822.0


In [40]:
pd.DataFrame(dfAll.drop_duplicates(subset=['Id'], keep='last'))

,Date,Tweet Treated,Tweet Raw,User,Url,Id,Likes,Views
0,2022-01-01 18:54:29+00:00,president joe biden will speak with ukrainian ...,President Joe Biden will speak with Ukrainian ...,cnni,https://twitter.com/cnni/status/14773525459578...,1477352545957863430,163,NaN
1,2022-01-01 01:55:07+00:00,biden says he warned putin of 'heavy price' ov...,Biden says he warned Putin of 'heavy price' ov...,Reuters,https://twitter.com/Reuters/status/14770960141...,1477096014129602561,178,NaN
2,2022-01-01 01:05:08+00:00,biden says he made it clear to putin that russ...,Biden says he made it clear to Putin that Russ...,Reuters,https://twitter.com/Reuters/status/14770834331...,1477083433184731137,458,NaN
0,2022-01-02 21:00:25+00:00,in a region where the shadow of war is everywh...,In a region where the shadow of war is everywh...,nytimesworld,https://twitter.com/nytimesworld/status/147774...,1477746627641286661,8,NaN
1,2022-01-02 14:30:07+00:00,bosnia is facing its gravest crisis since the ...,Bosnia is facing its gravest crisis since the ...,nytimesworld,https://twitter.com/nytimesworld/status/147764...,1477648401785405440,13,NaN
...,...,...,...,...,...,...,...,...
4,2023-04-29 13:23:02+00:00,ukraine war: crimea oil tank set ablaze by rep...,Ukraine war: Crimea oil tank set ablaze by rep...,BBCWorld,https://twitter.com/BBCWorld/status/1652302472...,1652302472793210883,435,242940.0
5,2023-04-29 09:14:34+00:00,ukraine war: crimea oil tank set ablaze by rep...,Ukraine war: Crimea oil tank set ablaze by rep...,BBCWorld,https://twitter.com/BBCWorld/status/1652239945...,1652239945610739713,456,282937.0
6,2023-04-29 17:35:00+00:00,ukraine held military drills in an undisclosed...,Ukraine held military drills in an undisclosed...,Reuters,https://twitter.com/Reuters/status/16523658814...,1652365881467445248,82,76332.0
7,2023-04-29 04:30:00+00:00,after a year-long war of words between florida...,After a year-long war of words between Florida...,Reuters,https://twitter.com/Reuters/status/16521683298...,1652168329845436416,109,71822.0


In [ ]:
dfAll.to_csv("completeDatasetNews.csv", index=False, quoting=csv.QUOTE_NONNUMERIC)